# **Lab Task 10 (Skeleton Code)**

In [1]:
import numpy as np

# Initialize an empty 6x7 Connect-4 board
def init_board():
    """
    Initialize the Connect-4 board.
    The board is represented as a 6x7 grid where 0 represents an empty space,
    1 represents the human player, and 2 represents the AI player.
    """
    # TODO: Implement this function

    return np.zeros((6, 7), dtype=int)




In [2]:

# Print the board in a readable format
def display_board(board):
    """
    Print the Connect-4 board to the console in a readable format.
    The board is printed from the bottom row to the top.
    """
    # TODO: Implement this function

    print(" 0 1 2 3 4 5 6")
    print("---------------")
    for row in range(5, -1, -1):
        print("|", end="")
        for col in range(7):
            if board[row][col] == 0:
                print(" ", end="|")
            elif board[row][col] == 1:
                print("X", end="|")
            else:
                print("O", end="|")
        print()
    print("---------------")
    print(" 0 1 2 3 4 5 6")



In [3]:
# Get a list of valid columns where a move can be made
def available_columns(board):
    """
    Generate a list of valid columns where a piece can be dropped.
    A column is valid if the topmost space is empty.
    """
    # TODO: Implement this function

    valid_cols = []
    for col in range(7):
        if board[5][col] == 0:
            valid_cols.append(col)
    return valid_cols


In [4]:
# Drop a piece into the board
def drop_piece(board, column, player):
    """
    Drop the piece (either 1 for human or 2 for AI) into the specified column.
    The piece should fall to the first available empty row in that column.
    """
    # TODO: Implement this function

    board_copy = board.copy()

    # Find the first empty row in the selected column (from bottom to top)
    for row in range(6):
        if board_copy[row, column] == 0:
            board_copy[row, column] = player
            return board_copy

    return board_copy



In [5]:
# Check for a winning condition
def check_winner(board, player):
    """
    Check if the specified player (1 or 2) has won the game.
    A player wins if they have four of their pieces in a row either:
        - Horizontally
        - Vertically
        - Diagonally (both left and right)
    """
    # TODO: Implement this function

    # Check horizontal win
    for row in range(6):
        for col in range(4):  # Only need to check starting positions 0-3
            if (board[row, col] == player and
                board[row, col+1] == player and
                board[row, col+2] == player and
                board[row, col+3] == player):
                return True

    # Check vertical win
    for row in range(3):  # Only need to check starting positions 0-2
        for col in range(7):
            if (board[row, col] == player and
                board[row+1, col] == player and
                board[row+2, col] == player and
                board[row+3, col] == player):
                return True

    # Check diagonal win (bottom-left to top-right)
    for row in range(3):  # Only need to check starting positions 0-2
        for col in range(4):  # Only need to check starting positions 0-3
            if (board[row, col] == player and
                board[row+1, col+1] == player and
                board[row+2, col+2] == player and
                board[row+3, col+3] == player):
                return True

    # Check diagonal win (top-left to bottom-right)
    for row in range(3, 6):  # Only need to check starting positions 3-5
        for col in range(4):  # Only need to check starting positions 0-3
            if (board[row, col] == player and
                board[row-1, col+1] == player and
                board[row-2, col+2] == player and
                board[row-3, col+3] == player):
                return True

    return False

In [6]:
# Evaluate the board state and return a numerical score
def score_position(board, player) :
    """
    Evaluate the current board state for the given player.
    If the player has won, return a high positive value.
    If the opponent has won, return a high negative value.
    Otherwise, return 0 or a heuristic evaluation score.

    Hint for implementation:
    - If the player has won (check for 4-in-a-row in any direction), return a high positive score.
    - If the opponent has won, return a high negative score.
    - If no one has won, evaluate the current board based on the player's advantage:
        - Check the number of 3-in-a-row or 2-in-a-row formations.
        - Consider the number of potential winning opportunities.
        - Penalize for blocking the opponent's possible winning moves.
    - You could assign values to the different patterns (e.g., 3-in-a-row with space could get +10).
    """
    # TODO: Implement this function

    opponent = 1 if player == 2 else 2

    # Check for immediate win/loss
    if check_winner(board, player):
        return 1000000  # Player has won
    if check_winner(board, opponent):
        return -1000000  # Opponent has won

    score = 0

    # Score center column (strategic advantage)
    center_array = board[:, 3]
    center_count = np.count_nonzero(center_array == player)
    score += center_count * 3

    # Check horizontal windows
    for row in range(6):
        for col in range(4):
            window = board[row, col:col+4]
            score += evaluate_window(window, player, opponent)

    # Check vertical windows
    for col in range(7):
        for row in range(3):
            window = board[row:row+4, col]
            score += evaluate_window(window, player, opponent)

    # Check diagonal windows (bottom-left to top-right)
    for row in range(3):
        for col in range(4):
            window = [board[row+i, col+i] for i in range(4)]
            score += evaluate_window(window, player, opponent)

    # Check diagonal windows (top-left to bottom-right)
    for row in range(3, 6):
        for col in range(4):
            window = [board[row-i, col+i] for i in range(4)]
            score += evaluate_window(window, player, opponent)

    return score

def evaluate_window(window, player, opponent):
    """
    Helper function to evaluate a window of 4 positions.
    """
    score = 0

    # Count pieces in the window
    player_count = np.count_nonzero(window == player)
    opponent_count = np.count_nonzero(window == opponent)
    empty_count = np.count_nonzero(window == 0)

    # Score the window based on its contents
    if player_count == 4:
        score += 100  # This should not happen normally as it would be caught by check_winner
    elif player_count == 3 and empty_count == 1:
        score += 5  # Three in a row with a possibility to win
    elif player_count == 2 and empty_count == 2:
        score += 2  # Two in a row with possibilities

    # Penalize opponent's potential wins
    if opponent_count == 3 and empty_count == 1:
        score -= 4  # Block opponent's potential win

    return score




In [7]:
# Minimax algorithm with Alpha-Beta Pruning
def minimax(board, depth, alpha, beta, maximizing_player):
    """
    Minimax algorithm with Alpha-Beta pruning to choose the best move.
    If it's the AI's turn (maximizing player), it will maximize the evaluation score.
    If it's the human's turn (minimizing player), it will minimize the evaluation score.
    """
    # TODO: Implement this function using recursion and alpha-beta pruning.

    valid_locations = available_columns(board)

    # Terminal conditions: game over or maximum depth reached
    is_terminal = len(valid_locations) == 0 or check_winner(board, 1) or check_winner(board, 2) or depth == 0
    if is_terminal:
        if check_winner(board, 2):  # AI wins
            return (None, 1000000)
        elif check_winner(board, 1):  # Human wins
            return (None, -1000000)
        elif len(valid_locations) == 0:  # Game is a draw
            return (None, 0)
        else:  # Maximum depth reached
            return (None, score_position(board, 2))

    if maximizing_player:  # AI's turn (player 2)
        value = float('-inf')
        column = valid_locations[0]  # Default move

        for col in valid_locations:
            # Simulate dropping a piece
            board_copy = drop_piece(board, col, 2)

            # Recursive call
            new_score = minimax(board_copy, depth-1, alpha, beta, False)[1]

            # Update best value and move
            if new_score > value:
                value = new_score
                column = col

            # Alpha-Beta pruning
            alpha = max(alpha, value)
            if alpha >= beta:
                break

        return column, value

    else:  # Human's turn (player 1)
        value = float('inf')
        column = valid_locations[0]  # Default move

        for col in valid_locations:
            # Simulate dropping a piece
            board_copy = drop_piece(board, col, 1)

            # Recursive call
            new_score = minimax(board_copy, depth-1, alpha, beta, True)[1]

            # Update best value and move
            if new_score < value:
                value = new_score
                column = col

            # Alpha-Beta pruning
            beta = min(beta, value)
            if alpha >= beta:
                break

        return column, value


In [8]:
# Choose the best move for the AI
def best_move(board):
    """
    Calculate the best move for the AI player using the Minimax algorithm.
    It should search for the best column based on the evaluation function.
    """
    # TODO: Implement this function

    column, minimax_score = minimax(board, 4, float('-inf'), float('inf'), True)
    return column


In [9]:
# Human player makes a move
def human_move(board):
    """
    Allow the human player to choose a valid move (column number).
    Ensure that the column is valid and not full.
    """
    # TODO: Implement this function

    valid_columns = available_columns(board)

    if not valid_columns:
        return None  # No valid moves available

    column = -1
    while column not in valid_columns:
        try:
            column = int(input("Your turn! Choose a column (0-6): "))
            if column < 0 or column > 6:
                print("Column must be between 0 and 6.")
                column = -1
            elif column not in valid_columns:
                print(f"Column {column} is full. Try another column.")
        except ValueError:
            print("Please enter a valid integer.")

    return column


In [10]:
# Run a simulation where Human plays against the AI
def play_game():
    """
    Run the Connect-4 game simulation where the human player and AI take turns.
    Game Loop:
    - Print the board before each turn.
    - The human player selects a valid column and drops a piece.
    - Check if the human has won.
    - AI chooses the best move using Minimax and drops a piece.
    - Check if the AI has won.
    - Repeat until there is a winner or the board is full.
    """
    board = init_board()
    game_over = False
    turn = 0  # 0 for human, 1 for AI

    print("Welcome to Connect-4!")
    print("You are X, and the AI is O.")

    while not game_over:
        display_board(board)

        # Human's turn
        if turn == 0:
            col = human_move(board)
            if col is None:
                print("No valid moves available. Game is a draw!")
                game_over = True
                continue

            board = drop_piece(board, col, 1)

            if check_winner(board, 1):
                display_board(board)
                print("Congratulations! You win!")
                game_over = True

        # AI's turn
        else:
            print("AI is thinking...")
            col = best_move(board)
            if col is None:
                print("No valid moves available. Game is a draw!")
                game_over = True
                continue

            board = drop_piece(board, col, 2)
            print(f"AI dropped a piece in column {col}")

            if check_winner(board, 2):
                display_board(board)
                print("AI wins! Better luck next time.")
                game_over = True

        # Check if the board is full (draw)
        if len(available_columns(board)) == 0 and not game_over:
            display_board(board)
            print("The game is a draw!")
            game_over = True

        # Switch turns
        turn = 1 - turn



In [ ]:
# Start the game
if __name__ == "__main__":
    play_game()


Welcome to Connect-4!
You are X, and the AI is O.
 0 1 2 3 4 5 6
---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------
 0 1 2 3 4 5 6
Please enter a valid integer.
 0 1 2 3 4 5 6
---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|X| | | | | | |
---------------
 0 1 2 3 4 5 6
AI is thinking...
AI dropped a piece in column 3
 0 1 2 3 4 5 6
---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|X| | |O| | | |
---------------
 0 1 2 3 4 5 6
 0 1 2 3 4 5 6
---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|X| | | | | | |
|X| | |O| | | |
---------------
 0 1 2 3 4 5 6
AI is thinking...
AI dropped a piece in column 0
 0 1 2 3 4 5 6
---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
|O| | | | | | |
|X| | | | | | |
|X| | |O| | | |
---------------
 0 1 2 3 4 5 6
 0 1 2 3 4 5 6
---------------
|